In [1]:
import sys

sys.path.append('../GSL')

In [2]:
import os
from glob import glob
import yaml
from easydict import EasyDict as edict
import pickle

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import numpy as np
from sklearn.preprocessing import StandardScaler
import random

from torch_geometric.utils import erdos_renyi_graph, dense_to_sparse, to_dense_adj
from torch_geometric_temporal.signal import temporal_signal_split

from dataset.make_dataset_ecl import ECLDatasetLoader
from dataset.make_dataset_ett import ETTDatasetLoader # ETT -> 시간 정보
from dataset.make_dataset_exchange import ExchangeDatasetLoader
from dataset.make_dataset_wth import WTHDatasetLoader
from dataset.make_dataset_covid19 import COVID19DatasetLoader
from dataset.make_dataset_traffic import TrafficDatasetLoader

from dataset.utils import download_file, Info

from utils.scalers import Scaler
import gdown
from dataclasses import dataclass

In [4]:
covid = pd.read_csv('../GSL/data/COVID19/time_series_covid19_confirmed_global.csv')

In [24]:
covid

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,7/5/22,7/6/22,7/7/22,7/8/22,7/9/22,7/10/22,7/11/22,7/12/22,7/13/22,7/14/22
0,NaN,Afghanistan,33.939110,67.709953,0,0,0,0,0,0,...,182793,182979,183084,183221,183235,183265,183268,183272,183285,183358
1,NaN,Albania,41.153300,20.168300,0,0,0,0,0,0,...,282690,283811,284758,285731,286732,287984,288176,289391,290954,290954
2,NaN,Algeria,28.033900,1.659600,0,0,0,0,0,0,...,266173,266181,266202,266228,266246,266257,266274,266303,266328,266356
3,NaN,Andorra,42.506300,1.521800,0,0,0,0,0,0,...,44177,44671,44671,44671,44671,44671,44671,44671,44671,44671
4,NaN,Angola,-11.202700,17.873900,0,0,0,0,0,0,...,101320,101320,101320,101320,101320,101320,101320,101320,101320,101600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280,NaN,West Bank and Gaza,31.952200,35.233200,0,0,0,0,0,0,...,662490,662490,662490,662490,662490,662490,662490,662588,662588,662588
281,NaN,Winter Olympics 2022,39.904200,116.407400,0,0,0,0,0,0,...,535,535,535,535,535,535,535,535,535,535
282,NaN,Yemen,15.552727,48.516388,0,0,0,0,0,0,...,11832,11832,11832,11832,11832,11832,11832,11832,11832,11832
283,NaN,Zambia,-13.133897,27.849332,0,0,0,0,0,0,...,326259,326408,326651,326794,326794,327052,327102,327102,327102,327102


In [6]:
country = ['US','Canada','Mexico','Russia',
           'United Kingdom','Italy','Germany','France',
           'Belarus','Brazil','Peru','Ecuador',
           'Chile','India','Turkey','Saudi Arabia',
           'Pakistan','Iran','Singapore','Qatar',
           'Bangladesh','United Arab Emirates','China','Japan','Korea, South']


In [7]:
my_covid = covid[covid['Country/Region'].isin(country)].copy()

In [8]:
my_covid.drop('Province/State', axis=1, inplace = True)

In [9]:
my_covid_confimed = my_covid[my_covid.columns.tolist()[:1]+my_covid.columns.tolist()[3:]].copy()
my_covid_confimed.reset_index(drop=True, inplace=True)

In [10]:
temp = pd.to_datetime(my_covid.columns[3:])

In [11]:
my_covid_confimed.columns.values[1:] = temp

In [12]:
df = my_covid_confimed.groupby('Country/Region').sum().copy()

In [13]:
total_confirmed = df.sum(axis=1)

In [14]:
country_lat_long_dict = {}

for cnt in country:
    lat = my_covid[covid['Country/Region']==cnt][['Lat', 'Long']].values
#     long = my_covid[covid['Country/Region']==cnt]['Long'].values

    country_lat_long_dict[cnt] = lat

/tmp/ipykernel_47879/4011621278.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  lat = my_covid[covid['Country/Region']==cnt][['Lat', 'Long']].values


In [15]:
multi_region=[]

for cnt in country_lat_long_dict.keys():
    if len(country_lat_long_dict[cnt]) != 1:
        multi_region.append(cnt)

for key in multi_region:
    country_lat_long_dict.pop(key)

In [16]:
country_lat_long_dict['Canada'] = np.array([[45.424721, -75.695000]])
country_lat_long_dict['United Kingdom'] = np.array([[51.509865, -0.118092]])
country_lat_long_dict['France'] = np.array([[48.864716, 2.349014]])
country_lat_long_dict['China'] = np.array([[39.916668, 116.383331]])

In [17]:
for cnt in country_lat_long_dict.keys():
    country_lat_long_dict[cnt] = [float(country_lat_long_dict[cnt][0][0]), float(country_lat_long_dict[cnt][0][1])]

In [18]:
import folium

In [19]:
m = folium.Map(location=[20,0], tiles="OpenStreetMap", zoom_start=2)

for cnt in country_lat_long_dict:
    folium.Circle(
        location=[country_lat_long_dict[cnt][0], country_lat_long_dict[cnt][1]],
        popup=cnt,
        radius = float(total_confirmed[cnt])/20000,
        color = 'crimson',
        fill = True,
        fill_color = 'crimson').add_to(m)
    
#     folium.Marker(location=[country_lat_long_dict[cnt][0], country_lat_long_dict[cnt][1]],
#                 popup=cnt,
#                 icon=folium.DivIcon(html=f"""<div style="font-family: courier new; color: crimson">{cnt}</div>""")
#                 ).add_to(m)
    

# folium.PolyLine(locations=list(country_lat_long_dict.values()),
#                tooltip='Polyline',
#                color = 'crimson',
#                weight=1,
#                dash_array='10').add_to(m)

m.save('./map.html')
m